## Pythagorean Pentatonic Scales

This is an example of using PyTuning in a Jupyter notebook environment. It is an exploration of
possible pentatonic scales drawn from Pythagorean tuning.

### Imports

In [ ]:
from __future__ import division, print_function

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sympy as sp

from pytuning.scales.pythagorean import create_pythagorean_scale
from pytuning.scales.edo import create_edo_scale
from pytuning.utilities import ratio_to_cents, mask_scale, distinct_intervals, compare_two_scales, note_number_to_freq
from pytuning.metrics import sum_p_q_for_all_intervals
from pytuning.tuning_tables import create_scala_tuning, create_timidity_tuning
from pytuning.scale_creation import find_best_modes
from pytuning.visualizations import consonance_matrix

%matplotlib inline
sns.set()
plt.ioff()

sp.init_printing(use_unicode=True)

from IPython.display import display

import pprint
pp = pprint.PrettyPrinter(indent=2)

The Pythagorean scale is created by walking up a circle of perfect fifths. However, one can also move *down* a fourth, and this creates some ambiguity to what the scale is. So, for example, if we just use up fifths:

In [ ]:
scale = create_pythagorean_scale(number_down_fifths=0)
scale

On the other hand, if we just use *down* fifths:

In [ ]:
scale = create_pythagorean_scale(number_down_fifths=11)
scale

(In the Wikipedia article the scale presented looks like):

In [ ]:
create_pythagorean_scale(scale_size=13, number_down_fifths=6)

(with both the augmented fourth and diminished fifth, which is why we set scale_size to 13.)

### The difference between up and down

Let's take a look at the two extremes, and see how much they differ

In [ ]:
up_scale   = create_pythagorean_scale(number_down_fifths=0)
down_scale = create_pythagorean_scale(number_down_fifths=11)
display(up_scale)
display(down_scale)

Looking at the differences in the degrees, we see that all degrees differ from their siblings by the
same amount:

In [ ]:
pp.pprint([ratio_to_cents(x) for x in np.array(up_scale) / np.array(down_scale)])

In [ ]:
np.array(up_scale)

Rationally, this is $\frac{531441}{524288}$):

In [ ]:
display([x for x in np.array(up_scale) / np.array(down_scale)])

(or, as a ratio:)

In [ ]:
sp.Rational(531441,524288).evalf()

Looking at it this way, we have 12 different scales:

In [ ]:
for x in range(12):
    display(create_pythagorean_scale(number_down_fifths=x))

### Consonance

Let's see if these scales vary in dissonance. For this we'll use the `sum_p_q_for_all_intervals` metric.

To find out we'll create `dict`, with the key as the number of down-facing fifths, and the value
the actual scale:

In [ ]:
pythag_dict = {}
for x in range(12):
    pythag_dict[x] =  create_pythagorean_scale(number_down_fifths=x)

In [ ]:
pythag_dict

Now map the metric

In [ ]:
for key in pythag_dict:
    pythag_dict[key] = list(sum_p_q_for_all_intervals(pythag_dict[key]).values())[0]
    
pp.pprint(pythag_dict)

They all have the same rating.

But what happens if we just examine a single mode, say, one of the pentatonic scales? Let's take the Major
Pentatonic:

$$
\left[ 2, 2, 3, 2, 3 \right]
$$

and see.

Note that we hash the mode mask against all the possible Pythagorean scales, and because we're only taking
selected notes, we can at times have repetitions. The following function strips out those repetitions
(mode scales which are the same owing to the fact that there are no differences in the set of selected
tones):

In [ ]:
def calculate_metric_with_mask(mask):
    all_scales = []
    pythag_dict = {}
    for x in range(12):
        try_scale = mask_scale(create_pythagorean_scale(number_down_fifths=x), mask)
        if try_scale not in all_scales:
            pythag_dict[x] =  try_scale
            all_scales.append(try_scale)

    metric_dict = {}
    for key in pythag_dict:
        metric_dict[key] = list(sum_p_q_for_all_intervals(pythag_dict[key]).values())[0]

    return pythag_dict, metric_dict

In [ ]:
major_pentatonic_mask = (0,2,4,7,9,12)
scales, metrics = calculate_metric_with_mask(major_pentatonic_mask)
display(scales)
display(metrics)

The metric is smallest for scale 0:

In [ ]:
scales[0]

This is without doubt the one that the standard would produce:

In [ ]:
mask_scale(create_pythagorean_scale(), major_pentatonic_mask)

Let's try the same thing with the Minor pentatonic scale:

$$
\left[ 3, 2, 2, 3, 2 \right]
$$

In [ ]:
minor_pentatonic_mask = (0,3,5,7,10,12)
scales, metrics       = calculate_metric_with_mask(minor_pentatonic_mask)
display(scales)
display(metrics)

Minimized for 3:

In [ ]:
scales[3]

Again, the standard tuning:

In [ ]:
mask_scale(create_pythagorean_scale(), minor_pentatonic_mask)

Interestingly, both the minor and major scale have the same set of distinct intervals (which explains why the
metric was the same value):

In [ ]:
minor_pent = mask_scale(create_pythagorean_scale(), minor_pentatonic_mask)
major_pent = mask_scale(create_pythagorean_scale(), major_pentatonic_mask)
display(sorted(distinct_intervals(minor_pent)))
display(sorted(distinct_intervals(major_pent)))

### Comparison with 12-EDO

Let's compare these with the 12-TET equivalents

In [ ]:
major_edo = mask_scale(create_edo_scale(12), major_pentatonic_mask)
minor_edo = mask_scale(create_edo_scale(12), minor_pentatonic_mask)
display(major_edo)
display(minor_edo)

In [ ]:
print(compare_two_scales(minor_edo, minor_pent, title=['Minor EDO', 'Minor Pythagorean']))
print()
print(compare_two_scales(major_edo, major_pent, title=['Major EDO', 'Major Pythagorean']))

### Which Pentatonic Scales Sounds the Best?

Let's take the standard Pythagorean scale and perform a consonance search.

We'll use `find_best_modes()` and select five tones. We'll also set `num_scales` to `None` so that
all scales will be returned.

In [ ]:
standard_pythag = create_pythagorean_scale()
mode_objects = find_best_modes(standard_pythag, 5, ['sum_p_q_for_all_intervals'], num_scales=None,
                              metric_function=sum_p_q_for_all_intervals)

In [ ]:
display(mode_objects[0]['scale'])

.... which looks a lot like the major pentatonic:

In [ ]:
display(major_pent)

Let's look at the top few scales, to see if the minor is up there too....

In [ ]:
top_scales = [ x['scale'] for x in mode_objects][:5]
for s in top_scales:
    display(s)

It's number 4 on the list:

In [ ]:
display(minor_pent)
display(mode_objects[3]['scale'])

But this is a bit misleading, as if we actually look at the scores, you'll see that the top five all have 
the same score:

In [ ]:
display([x['sum_p_q_for_all_intervals'] for x in mode_objects][:26])

Thus by this metric the following scales are all equivalently "good":

In [ ]:
for x in range(5):
    display(mode_objects[x]['scale'])

Let's do a visualization of of one of the scale, the last one, for example:

In [ ]:
consonance_matrix(mode_objects[4]['scale'])

Interestingly, the scale patterns (the steps between adjacent degrees) for all five of these scales are
rotational permutations of one another:

In [ ]:
for x in range(5):
    display(mode_objects[x]['steps'])

But owing to the fact that the scale itself is not equally tempered, the scales themselves are *not*.

If you wanted to open things up a bit, the next five have the same value too. (Note that including
10 scales in the list is somewhat arbitrary: the first five have the same metric, but the next *20* are
grouped together. Just taking 5 out of the 20 is arbitrary.)

So, the top 10 scales -- the first five of which fall into a consonance group, the second of which fall into their own group:

In [ ]:
for x in range(10):
    display(mode_objects[x]['scale'])

### Tuning

For working with these tunings, the following function will take an index into the `mode_objects` data
structure and display several different tuning files for the resultant scale:

In [ ]:
def create_tunings(index, mode_objects = mode_objects, reference_note = 69):
    reference_frequency = note_number_to_freq(reference_note, create_edo_scale(12))
    steps               = mode_objects[index]['steps']
    scale               = mode_objects[index]['scale']
    original_scale      = mode_objects[index]['original_scale']
    mode_steps          = mode_objects[index]['steps']
    mask                = mode_objects[index]['mask']
    metric               = mode_objects[index]['sum_p_q_for_all_intervals']
    em_spec = "set scale "  + str(tuple(mode_objects[10]['steps'])) + " %d" % reference_note
    timidity = create_timidity_tuning(scale, reference_note)
    
    print("*******************")
    print("Summary Information")
    print("*******************")
    print("Reference Note:      %d" % reference_note)
    print("Reference Frequency: %f" % reference_frequency)
    print("Metric:              %d" % metric)
    print("Mode Steps:          %s" % str(mode_steps))
    print("Mode Mask:           %s" % str(mask))
    print("")
    print("********")
    print("Tunings")
    print("********")
    
    print("\n***Emergent Tuning***\n")
    print(em_spec)
    print("\n***Scala Tuning***\n")
    print(create_scala_tuning(original_scale, "Pentatonic %d" % index))
    print("\n***Timidity Tuning***\n")
    print(timidity)
    

As an example, the 10th scale, referenced to C = 261.6 Hz:

In [ ]:
create_tunings(9, reference_note=60)